In [48]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import country_converter as coco


## Get the data
The function bellow is used to extract the first table present in a wikipedia Page

In [49]:

def getDataFrameFromWikipedia(wikipedia_url, table_no=1):
    """
    returns a dataframe of the data from the wikipedia page. \n
    optionaly, if there are multiple tables on the page, you can specify which table to use in `table_no`.
    """
    response = requests.get(wikipedia_url)
    print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.findAll('table', {'class': 'wikitable'})

    if table_no >= 1 and table_no <= len(tables):
        df = pd.read_html(str(tables[table_no - 1]))[0]
        return df
    else:
        return None


## Normalise the country
- As there are no standerdised way to get countries, we use a library called `country_converter` to convert countries for us.
- Some country might not exist (Channel Islands, European Union, World...), in that case, we just drop the whole record for this country

In [50]:
def normaliseCountryNames(df):
    df.columns.values[0] = 'Country'
    df = df.fillna('')
    df['Country'] = df['Country'].apply(lambda x: coco.convert(names=x, to='name_short', not_found='not found'))
    df['Country'] = df['Country'].str.rstrip('*')
    df = df[df['Country'] != 'not found']
    return df


In [51]:
def keepOnlyNumberInLastColumn(df):
    last_col = df.columns[-1]
    df.loc[:, last_col] = df[last_col].str.extract('(\d+)')
    df.loc[:, last_col] = pd.to_numeric(df[last_col], errors='coerce')

    return df


## All column

In [52]:
the_final_table = []


In [53]:
## GET GDP PER CAPITA

gdp_per_capita = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita")

df= gdp_per_capita

df = df.drop(["IMF[4][5]", "United Nations[7]"], axis = 1)
df = df.drop(0)
df.columns = df.columns.droplevel(-1)
df.columns = [*df.columns[:-1], 'years']
df.columns.values[0] = 'Country'

df = df.drop(["UN Region", "years"], axis = 1)


df = normaliseCountryNames(df)

the_final_table = df

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/3674213559.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["IMF[4][5]", "United Nations[7]"], axis = 1)
Channel Islands not found in regex
European Union[n 1] not found in regex
World not found in regex
Zanzibar not found in regex


           Country World Bank[6]
1           Monaco        234317
2    Liechtenstein        184083
3       Luxembourg        126426
4          Bermuda        118846
5          Ireland        104039
..             ...           ...
219     Madagascar           505
220    South Sudan          1072
221   Sierra Leone           461
222    Afghanistan           364
223        Burundi           238

[219 rows x 2 columns]


In [54]:
# List of countries by Internet connection speeds, Fixed broadband, Average download speed (Mbit/s) (Ookla).

internet_speed = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_Internet_connection_speeds")

df = internet_speed
df = df.drop(["Rank", "Averagedownloadspeed(Mbit/s)(M-Lab)[2]", "Averagedownloadspeed(Mbit/s)(SpeedTestNet.io)", "Averagedownloadspeed(Mbit/s)(Speed-Test-Pros.com)[4]"], axis = 1)
df.columns = [*df.columns[:-1], "Average download speed (Mbit/s)"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


             Country  Average download speed (Mbit/s)
0            Romania                           178.90
1        South Korea                           241.58
2          Hong Kong                           265.17
3             Monaco                           220.35
4          Singapore                           259.11
..               ...                              ...
96          Dominica                            37.62
97   North Macedonia                            37.20
98            Greece                            36.73
99        Bangladesh                            36.02
100          Senegal                            35.32

[101 rows x 2 columns]


In [55]:
# List of countries by alcohol consumption per capita, Recorded per capita consumption of pure alcohol (litres) per adult 15 years of age and over per year, 2016.
alcohol_consumption = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_alcohol_consumption_per_capita")

df = alcohol_consumption

df = df.drop(["1996[7]"], axis = 1)
df.columns = [*df.columns[:-1], "Alcohiol consumption per capita (litres) 2016"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Netherlands Antilles not found in regex


         Country Alcohiol consumption per capita (litres) 2016
0    Afghanistan                                           0.2
1        Albania                                           7.5
2        Algeria                                           0.9
3        Andorra                                          11.3
4         Angola                                           6.4
..           ...                                           ...
186    Venezuela                                           5.6
187      Vietnam                                           8.3
188        Yemen                                           0.1
189       Zambia                                           4.8
190     Zimbabwe                                           4.8

[190 rows x 2 columns]


In [56]:
# List of countries by intentional homicide rate, Intentional homicide victims per 100,000 inhabitants. From UNODC, rate.

homicide_rate = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate",2)

df = homicide_rate

df = df.drop(["Region", "Subregion", "Year", "Count"], axis = 1)
df.columns = [*df.columns[:-1], "Intentional homicide victims per 100,000 inhabitants"]
df['Location'] = df['Location'].str.replace('*', '', regex=False).str.strip()
df = df.rename(columns={'Location': 'Country'})
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
 not found in regex


200


Northern Ireland not found in regex
Scotland not found in regex


            Country Intentional homicide victims per 100,000 inhabitants
1       Afghanistan                                                4.0  
2           Albania                                                2.3  
3           Algeria                                                1.6  
4    American Samoa                                                0.0  
5           Andorra                                                2.6  
..              ...                                                ...  
202       Venezuela                                               19.3  
203         Vietnam                                                1.5  
204           Yemen                                                6.3  
205          Zambia                                                5.2  
206        Zimbabwe                                                6.1  

[204 rows x 2 columns]


In [57]:
# List of countries by military expenditures, List by the Stockholm International Peace Research Institute, % of GDP.

list_of_military_expendature = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_with_highest_military_expenditures", 1)

df = list_of_military_expendature

df = df[['Country', '% of GDP']]
df.columns = [*df.columns[:-1], "Military Expenditure % GDP"]

normaliseCountryNames(df)
the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df)


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
World total not found in regex


200
               Country  Military Expenditure % GDP
0          World total                         2.2
1        United States                         3.5
2             China[a]                         1.6
3               Russia                         4.1
4                India                         2.4
5   Saudi Arabia[a][b]                         7.4
6       United Kingdom                         2.2
7              Germany                         1.4
8            France[c]                         1.9
9          South Korea                         2.7
10               Japan                         1.1
11             Ukraine                        34.0
12               Italy                         1.7
13           Australia                         1.9
14              Canada                         1.2
15              Israel                         4.5
16               Spain                         1.5
17              Brazil                         1.1
18           Poland[c]     

In [58]:
## GET Human Development Index,

human_dev_index = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index", 2)

df= human_dev_index
# print(df)


df = df.drop(["Rank"], axis=1)

df = df[[('Nation', 'Nation'), ('HDI', '2021 data (2022\xa0report)\u200b[2]')]]
df.columns = df.columns.droplevel(0)

df.columns = [*df.columns[:-1], "Human Development Index"]
df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))





200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/1365520217.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Rank"], axis=1)


                      Country  Human Development Index
0                 Switzerland                    0.962
1                      Norway                    0.961
2                     Iceland                    0.959
3                   Hong Kong                    0.952
4                   Australia                    0.951
..                        ...                      ...
186                   Burundi                    0.426
187  Central African Republic                    0.404
188                     Niger                    0.400
189                      Chad                    0.394
190               South Sudan                    0.385

[191 rows x 2 columns]


In [59]:
# Democracy Index, Democracy Index, 2020

democracy_index = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/The_Economist_Democracy_Index", 4)

df= democracy_index
df = df[['Country','2020']]
df.columns = [*df.columns[:-1], "Democracy Index (2020)"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))



200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


           Country  Democracy Index (2020)
0           Canada                    9.24
1    United States                    7.92
2          Austria                    8.16
3          Belgium                    7.51
4           Cyprus                    7.56
..             ...                     ...
162       Tanzania                    5.10
163           Togo                    2.80
164         Uganda                    4.94
165         Zambia                    4.86
166       Zimbabwe                    3.16

[167 rows x 2 columns]


In [60]:
# List of countries by tertiary education attainment,Countries by level of tertiary education, at least a 2-year tertiary degree or its equivalent %.

tertiary_education_attainment = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_tertiary_education_attainment", 2)

df= tertiary_education_attainment
df = df[[('Country', 'Country'), ('Ages 25–34:\xa0% equivalent to a degree course lasting at least:', '2 years')]]
df.columns = [*df.columns[:-1], "tertiary education attainment, 2 y"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


           Country  tertiary education attainment, 2 y
0        Australia                                48.0
1          Austria                                39.0
2          Belgium                                43.0
3           Brazil                                16.0
4           Canada                                59.0
5            Chile                                36.0
6            China                                35.8
7         Colombia                                27.0
8       Costa Rica                                28.0
9          Czechia                                31.0
10         Denmark                                44.0
11         Estonia                                41.0
12         Finland                                41.0
13          France                                45.0
14         Germany                                30.0
15          Greece                                40.0
16         Hungary                                32.0
17        

In [61]:
# Importance of religion by country, Countries/Districts, Yes important.

importance_religion = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Importance_of_religion_by_country")

df= importance_religion
df = df.drop(["No, unimportant[1]"], axis=1)
df.columns = [*df.columns[:-1], "Importance of religion"]
df = normaliseCountryNames(df)
df = keepOnlyNumberInLastColumn(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


        Country Importance of religion
0        Sweden                     17
1       Denmark                     19
2       Estonia                     16
3         Japan                     24
4     Hong Kong                     24
..          ...                    ...
107   Sri Lanka                     99
108       Yemen                     99
109  Bangladesh                     99
110   Indonesia                     99
111       Niger                     99

[112 rows x 2 columns]


In [62]:
# Christianity by country, UN members and dependent territories, % Chriastian.

christianity_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Christianity_by_country", 3)

df= christianity_by_country
df = df[['Country or entity', '% Christian']]
df.columns = [*df.columns[:-1], "Importance of religion"]
df = normaliseCountryNames(df)
df = keepOnlyNumberInLastColumn(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Europe not found in regex
Latin America and the Caribbean not found in regex
Africa not found in regex
Asia not found in regex
North America not found in regex
Oceania not found in regex
Middle East-North Africa not found in regex
Total not found in regex


            Country Importance of religion
0       Afghanistan                    NaN
1           Albania                    NaN
2           Algeria                    NaN
3    American Samoa                    NaN
4           Andorra                    NaN
..              ...                    ...
190       Venezuela                    NaN
191         Vietnam                    NaN
192           Yemen                    NaN
193          Zambia                    NaN
194        Zimbabwe                    NaN

[195 rows x 2 columns]


In [63]:
#11. Islam by country, Muslim percentage of total population
islam_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Islam_by_country")
df = islam_by_country
df = df.drop(["Total Population", "Muslim Population", "Percentage of world (%)", "Sources"], axis = 1)
df = df.drop(["Unnamed: 6"], axis = 1)
df = df.rename(columns={'Country/Region': 'Country'})
df = normaliseCountryNames(df)
df = df.rename(columns={'Muslim percentage of total population': 'Muslim percentage of total population_islam'})

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')
print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Netherlands Antilles not found in regex


                       Country Muslim percentage of total population_islam
0                  Afghanistan                                        99.7
1                      Albania                                        58.8
2                      Algeria                                          99
3               American Samoa                                       < 0.1
4                      Andorra                                         2.6
..                         ...                                         ...
227  Wallis and Futuna Islands                                       < 0.1
228             Western Sahara                                        99.4
229                      Yemen                                        97.2
230                     Zambia                                           1
231                   Zimbabwe                                         0.7

[231 rows x 2 columns]


In [64]:
#12 Buddhism by country, Buddhist population by country, % population.
Buddhist_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Buddhism_by_country")

df = Buddhist_by_country
df = df.drop(["Pew estimates (2010)[1]", "Other estimates"], axis = 1)
df.columns = df.columns.droplevel(0)
df = df.drop(["Population", "No. of Buddhists", "Census Year"], axis = 1)
#print(df.columns)

df = df.rename(columns={'Country/Territory': 'Country'})
df = df.rename(columns={'% Buddhist': 'Buddhist population by country, % population'})
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/3932892944.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Pew estimates (2010)[1]", "Other estimates"], axis = 1)
World not found in regex


            Country Buddhist population by country, % population
0       Afghanistan                                             
1           Albania                                             
2           Algeria                                             
3    American Samoa                                             
4           Andorra                                             
..              ...                                          ...
174       Venezuela                                             
175         Vietnam                                             
176           Yemen                                             
177          Zambia                                             
178        Zimbabwe                                             

[179 rows x 2 columns]


In [65]:
#13 Jewish population by country, Table, Core population, pct
Jewish_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Jewish_population_by_country")
df = Jewish_by_country

df = df.drop(["Connected population", "Enlarged population","Eligible population","National official"], axis = 1)

df.columns = df.columns.droplevel(0)
df = df.drop(["Total", "pmp"], axis = 1)
df = df.rename(columns={'Countries': 'Country'})
df = df.rename(columns={'% pct': 'Core population, pct'})
#print(df.columns)

df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer', suffixes=('_final', '_jewish'))


print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/260437884.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Connected population", "Enlarged population","Eligible population","National official"], axis = 1)
Netherlands Antilles not found in regex
World not found in regex


                    Country  pct
0                    Israel   39
1             United States   35
2                    France  2.8
3                 Palestine  2.7
4                    Canada  2.4
..                      ...  ...
107                 Iceland    —
108              Montenegro    —
109  British Virgin Islands    —
110           Liechtenstein    —
111        Falkland Islands    —

[111 rows x 2 columns]


In [66]:
#14 List of countries by infant and under-five mortality rates,  Under-five mortality (deaths/1,000 live births) – 2019 estimates
mortality_rate = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_infant_and_under-five_mortality_rates",4)

df = mortality_rate
df = df.drop(["1950-1955", "1955-1960",'1960-1965', '1965-1970',
       '1970-1975', '1975-1980', '1980-1985', '1985-1990', '1990-1995',
       '1995-2000', '2000-2005', '2005-2010', '2010-2015', 'To CIA estimates'], axis = 1)
df = df.drop(0)

df = df.rename(columns={'Location [Note 1]': 'Country'})
df = df.rename(columns={'2015-2020': '2019 estimates'})

df['Country'] = df['Country'].str.rstrip(' *')

for index, row in df.iterrows():
    if '(' in row['Country'] and ')' in row['Country']:
        df.at[index, 'Country'] = row['Country'].split('(', 1)[1].split(')', 1)[0]

df.columns.values[0] = 'Country'


df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
disputed not found in regex
Congo  not found in regex
World not found in regex


                      Country  2019 estimates
1                     Iceland            1.25
2                       China            1.32
3                   Singapore            1.63
4                     Finland            1.71
5                       Japan            1.76
..                        ...             ...
197                      Mali           65.80
198                   Somalia           69.31
199                      Chad           74.50
200  Central African Republic           81.90
201              Sierra Leone           80.77

[199 rows x 2 columns]


In [67]:
#15 Age of criminal responsibility, By country, age reduced

Age= getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Age_of_criminal_responsibility")

df = Age


df = df.drop(["Age (full)[b]", "Ref","Notes"], axis = 1)

df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'Age (reduced)[a]': 'By country, age reduced'})

for index, row in df.iterrows():
    if '(' in row['Country'] and ')' in row['Country']:
        df.at[index, 'Country'] = row['Country'].split('(', 1)[1].split(')', 1)[0].strip()
    else:
        df.at[index, 'Country'] = row['Country'].strip()
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer', suffixes=('_final', '_age_reduced'))

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


         Country By country, age reduced
0    Afghanistan                      12
1        Albania                      14
2        Algeria                      13
3        Andorra                      12
4         Angola                      14
..           ...                     ...
165      Vanuatu                      10
166      Vietnam                      14
167        Yemen                       7
168       Zambia                       8
169     Zimbabwe                       7

[170 rows x 2 columns]


In [68]:
#16 List of countries by minimum wage, List, Annual, US$.
list = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage")
df = list

df = df.drop(["Minimum wage", "Workweek (hours)[4]","Hourly","Effective per"], axis = 1)


df.columns = df.columns.droplevel(0)

df = df.drop(["PPP (Int$)[7]"], axis = 1)
df = df.drop(0)
df = df.iloc[:, :-1]

df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'Nominal (US$)[6]': 'Annual US$'})
#print(df.columns)

df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/4045473643.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Minimum wage", "Workweek (hours)[4]","Hourly","Effective per"], axis = 1)


         Country Annual US$
1    Afghanistan      858.0
2        Albania     4637.0
3        Algeria     1777.0
4        Andorra    18253.0
5         Angola      663.0
..           ...        ...
197    Venezuela      10.32
198      Vietnam     1591.0
199        Yemen           
200       Zambia      596.0
201     Zimbabwe           

[201 rows x 2 columns]


In [69]:
#17 List of countries by external debt, List of countries with respect to external debt, % GDP
list_countries = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_external_debt")
df = list_countries


df = df.drop(["Per capita US dollars", "External debt US dollars","% of total wealth[1]","% of total wealth[1]","Date"], axis = 1)

df = df.rename(columns={'Country/Region': 'Country'})
df = df.rename(columns={'of GDP': 'external debt % GDP'})

for index, row in df.iterrows():
    if '(' in row['Country'] and ')' in row['Country']:
        df.at[index, 'Country'] = row['Country'].split('(', 1)[1].split(')', 1)[0]

df.columns.values[0] = 'Country'

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


            Country  % of GDP
0     United States    122.56
1    United Kingdom    287.08
2             Japan     98.44
3       Netherlands    381.62
4            France    112.06
..              ...       ...
205  United Kingdom      5.00
206          France      6.00
207  United Kingdom      2.00
208   Liechtenstein      0.00
209     New Zealand      0.00

[210 rows x 2 columns]


In [70]:
#18 List of countries by income equality, UN, World Bank and CIA list – income ratios and Gini indices, WB Gini %.
list = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_income_equality")
df = list

df = df.drop(["Subregion", "Region","UN R/P","CIA R/P[4]"], axis = 1)
df.columns = df.columns.droplevel(0)
df = df.drop(["Year"], axis = 1)
df = df.drop(0)


df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'%': 'WB Gini %'})
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/3041032724.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Subregion", "Region","UN R/P","CIA R/P[4]"], axis = 1)
European Union not found in regex
World not found in regex


       Country WB Gini %
1      Albania      29.4
2      Algeria      27.6
3       Angola      51.3
4    Argentina      42.0
5      Armenia      27.9
..         ...       ...
165  Venezuela      44.8
166    Vietnam      36.8
167      Yemen      36.7
168     Zambia      55.9
169   Zimbabwe      50.3

[168 rows x 2 columns]


In [71]:
#19. List of countries by total health expenditure per capita, Total health expenditure per capita in 2018 PPP international U.S. dollars, 2018.
list = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_total_health_expenditure_per_capita",2)
df = list

df = df.drop(["2019", "2020","2021"], axis = 1)
df = df.rename(columns={'Location': 'Country'})
df = df.rename(columns={'2018': 'Total health 2018'})

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Congo * not found in regex


                 Country  Total health 2018
0            Afghanistan                 71
1                Algeria                258
2                Andorra               3188
3                 Angola                 84
4    Antigua and Barbuda                850
..                   ...                ...
183              Vanuatu                102
184            Venezuela                165
185              Vietnam                164
186               Zambia                 75
187             Zimbabwe                115

[187 rows x 2 columns]


In [72]:
#20 List of countries by suicide rate, Suicide rates by sex and country, All.
list_suicide  = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_suicide_rate")
df = list_suicide

df = df.drop(["Male", "Female"], axis = 1)

df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'All': 'All Suicide rates by sex and country'})

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Africa not found in regex
Americas not found in regex
South-East Asia not found in regex
Europe not found in regex
Eastern Mediterranean not found in regex
Western Pacific not found in regex
Global not found in regex


                 Country  All Suicide rates by sex and country
0            Afghanistan                                   6.0
1                Albania                                   3.7
2                Algeria                                   2.6
3                 Angola                                  12.6
4    Antigua and Barbuda                                   0.3
..                   ...                                   ...
178            Venezuela                                   2.1
179              Vietnam                                   7.2
180                Yemen                                   7.1
181               Zambia                                  14.4
182             Zimbabwe                                  23.6

[183 rows x 2 columns]


In [73]:
# 21. List of sovereign states and dependencies by total fertility rate, 2019 List by the World Bank, 2019.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_total_fertility_rate", 3)

df = df.drop(["Rank"], axis = 1)
df = normaliseCountryNames(df)
df.fillna('')

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match for Mayotte (France)
More than one regular expression match

      Country Fertility rate in 2019 (births/woman)
0       Niger                                   6.8
1     Somalia                                   6.0
2    DR Congo                                   5.8
3        Mali                                   5.8
4        Chad                                   5.6
..        ...                                   ...
251  Pitcairn                                     -
252       NaN                                     -
253       NaN                                     -
254    Tuvalu                                     -
255   Vatican                                     -

[223 rows x 2 columns]


In [74]:
# 22. Tobacco consumption by country, Prevalence of current tobacco use among persons aged 15 years and older, 2000.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Tobacco_consumption_by_country")
df = normaliseCountryNames(df)
df = df.rename(columns={'Cigarettes': 'Annual Cigarette Consumption per person aged 15 or older, 2016'})

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Congo * not found in regex


                 Country  \
0                Andorra   
1             Luxembourg   
2                Belarus   
3        North Macedonia   
4                Albania   
..                   ...   
176  Antigua and Barbuda   
177                Ghana   
178           Mauritania   
179        Guinea-Bissau   
180    Brunei Darussalam   

     Annual Cigarette Consumption per person aged 15 or older, 2016  
0                                               6398.3               
1                                               6330.9               
2                                               2911.3               
3                                               2784.9               
4                                               2491.6               
..                                                 ...               
176                                               89.2               
177                                               40.5               
178                              

In [75]:
# 23. List of countries by obesity rate, list, 2016 %.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_obesity_rate")
df = normaliseCountryNames(df)
df = df.rename(columns={'Obesity rate (%)': 'Obesity rate(%), 2016'})

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


              Country  Obesity rate(%), 2016
0               Nauru                   61.0
1        Cook Islands                   55.9
2               Palau                   55.3
3    Marshall Islands                   52.9
4              Tuvalu                   51.6
..                ...                    ...
187          Cambodia                    3.9
188             India                    3.9
189       Timor-Leste                    3.8
190        Bangladesh                    3.6
191           Vietnam                    2.1

[192 rows x 2 columns]


In [76]:
# 24. List of countries by number of Internet users, Table, %.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_number_of_Internet_users", 3)
df.columns.values[0] = 'Country'
df = df[['Country', 'Pct']]
df = df.rename(columns={'Pct': 'Internet users (%)'})

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expression match for Macau, China
More than one regular expr

                       Country Internet users (%)
0                        China              76.4%
1                        India              62.6%
2                United States              92.4%
3                    Indonesia              78.8%
4                       Brazil              77.1%
..                         ...                ...
209                 St. Helena              53.8%
210           Falkland Islands              76.5%
211                 Montserrat              64.1%
212  Wallis and Futuna Islands              11.9%
213                       Niue              53.4%

[214 rows x 2 columns]


In [77]:
# 25. List of countries by median age, Median age per CIA World Factbook 2018 and 2020 (ranked) estimates, 2020 median combined.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_median_age")

df.columns = [' '.join(col).strip() for col in df.columns.values]
df.columns.values[0] = 'Country'
df = df[['Country', 'Median ages in years 2020 medians Combined']]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
European Union not found in regex
Virgin Islands not found in regex
World not found in regex


            Country  Median ages in years 2020 medians Combined
0       Afghanistan                                        19.5
1           Albania                                        34.3
2           Algeria                                        28.9
3    American Samoa                                        27.2
4           Andorra                                        46.2
..              ...                                         ...
224       Palestine                                        21.9
225  Western Sahara                                        21.5
227           Yemen                                        19.8
228          Zambia                                        16.9
229        Zimbabwe                                        20.5

[227 rows x 2 columns]


In [78]:
# 26. List of countries by economic freedom, 2019 Economic Freedom of the World Index, score.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_economic_freedom")
df = df[['Country', 'Score']]
df = df.rename(columns={'Score': 'Economic Freedom Score, 2019'})

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


            Country  Economic Freedom Score, 2019
0         Hong Kong                          90.2
1         Singapore                          89.4
2       New Zealand                          84.4
3       Switzerland                          81.9
4         Australia                          80.9
..              ...                           ...
175  Congo Republic                          39.7
176         Eritrea                          38.9
177            Cuba                          27.8
178       Venezuela                          25.9
179     North Korea                           5.9

[180 rows x 2 columns]


In [79]:
# 27. List of countries by oil production, per capita 2017. Valeur absente = 0, valeur manquante [–] = comme les autres.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_oil_production", 2)
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
World not found in regex


          Country  Oil production April 2022 (bbl/day)[1]
0   United States                                11884154
1    Saudi Arabia                                10644394
2          Russia                                10278370
3          Canada                                 4543800
4            Iraq                                 4470506
..            ...                                     ...
92         Taiwan                                     196
93       Slovakia                                      43
94        Morocco                                      25
95         Jordan                                      20
96          Spain                                      19

[97 rows x 2 columns]


In [80]:
# 28. List of countries by population growth rate, Table, UN 2015-20.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_population_growth_rate")
df.columns = [' '.join(col).strip() for col in df.columns.values]

df.columns.values[0] = 'Country'
df.columns.values[-1] = 'Population growth rate (%) (2015-2020)'
df = df.iloc[:, [0, -1]]
df = df.fillna('')

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
 not found in regex
World not found in regex
Asia/Other non-specified areas not found in regex


200


Congo * not found in regex
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands)
More than one regular expression match for Aruba * (Netherlands

                          Country Population growth rate (%) (2015-2020)
3                     Afghanistan                                   2.41
4                         Albania                                   0.13
5                         Algeria                                   1.67
6                         Andorra                                  -0.21
7                          Angola                                   3.28
..                            ...                                    ...
237        British Virgin Islands                                       
238  United States Virgin Islands                                       
239                           NaN                                       
240                     Palestine                                       
241                     Palestine                                       

[237 rows x 2 columns]


In [81]:
# 29. List of countries by life expectancy, List by the World Health Organization (2019), Life expectancy at birth, All.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy", 3)
df.columns = [' '.join(col).strip() for col in df.columns.values]
df.columns.values[0] = 'Country'
df.columns.values[1] = 'Life expectancy at birth, 2019'

df = df.iloc[:, [0, 1]]
df = df.fillna('')
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))



200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
 not found in regex
World not found in regex


                      Country Life expectancy at birth, 2019
1                       Japan                           84.3
2                 Switzerland                           83.4
3                 South Korea                           83.3
4                   Singapore                           83.2
5                       Spain                           83.2
..                        ...                            ...
180                Mozambique                           58.1
181                  Eswatini                           57.7
182                   Somalia                           56.5
183  Central African Republic                           53.1
184                   Lesotho                           50.7

[183 rows x 2 columns]


In [82]:
# 30. List of countries by meat consumption, Meat consumption by country, KG per capita 2002.

df = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_meat_consumption")
df.columns.values[0] = 'Country'
df.columns.values[1] = 'Meat consumption (kg per capita), 2002'
df = df.iloc[:, [0, 1]]
df = df.fillna('')

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Netherlands Antilles not found in regex


                 Country Meat consumption (kg per capita), 2002
0                Albania                                   38.2
1                Algeria                                   18.4
2         American Samoa                                   24.9
3                 Angola                                   25.0
4    Antigua and Barbuda                                   56.0
..                   ...                                    ...
182            Venezuela                                   56.6
183              Vietnam                                   28.6
184                Yemen                                   14.7
185               Zambia                                   11.9
186             Zimbabwe                                   15.2

[186 rows x 2 columns]


In [83]:
#31. List of countries by incarceration rate

incarceration_rate = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_incarceration_rate", 1)

df = incarceration_rate

df = df.drop("Counts[3]", axis=1)
df = df.drop(0)
df.columns = [*df.columns[:-1], "Incarceration rates (per 100 000)"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Republika Srpska not found in regex
Northern Ireland * [Note] not found in regex
Scotland * [Note] not found in regex


                          Country Incarceration rates (per 100 000)
1                     Afghanistan                                36
2                         Albania                               174
3                         Algeria                               217
4                  American Samoa                               538
5                         Andorra                                64
..                            ...                               ...
222        British Virgin Islands                               368
223  United States Virgin Islands                               345
224                         Yemen                                35
225                        Zambia                               118
226                      Zimbabwe                               121

[223 rows x 2 columns]


In [84]:
#32. List of countries by literacy rate

literacy_rate = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_literacy_rate", 2)

df = literacy_rate

df = df.drop([('Literacy rate[10][11]', 'Male'), ('Literacy rate[10][11]', 'Female'), ('Literacy rate[10][11]', 'Gender Gap'), ('Literacy rate[10][11]', 'Year')], axis=1)
df.columns = [*df.columns[:-1], "Literacy rate"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Congo * not found in regex
World not found in regex


         Country Literacy rate
0    Afghanistan         37.3%
1        Albania         98.1%
2        Algeria         81.4%
3        Andorra        100.0%
4         Angola         71.1%
..           ...           ...
190    Venezuela         97.1%
191      Vietnam         95.8%
192        Yemen         70.1%
193       Zambia         86.7%
194     Zimbabwe         86.5%

[194 rows x 2 columns]


In [85]:
#33. List of countries by age at first marriage

age_first_marriage = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_age_at_first_marriage")

df = age_first_marriage

df.drop(["Men", "Women", "Age Gap", "Age Ratio", "Year", "Source"], axis=1, inplace = True)
df.columns = [*df.columns[:-1], "Age at first marriage"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))
print("\n")


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Congo[a] not found in regex


                     Country  Age at first marriage
0                    Algeria                   30.3
1                     Angola                   23.1
2                      Benin                   21.3
3                   Botswana                   28.7
4               Burkina Faso                   21.1
5                    Burundi                   22.0
6                 Cabo Verde                   23.9
7                   Cameroon                   22.4
8   Central African Republic                   19.5
9                       Chad                   21.8
10                   Comoros                   23.0
12             Cote d'Ivoire                   23.1
13                  DR Congo                   21.6
14                  Djibouti                   28.9
15                     Egypt                   28.0
16         Equatorial Guinea                   23.3
17                   Eritrea                   24.9
18                  Eswatini                   28.5
19          

In [86]:
#34. List of countries by spending on education as percentage of GDP

spending_education = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_spending_on_education_as_percentage_of_GDP", 1)

df = spending_education

df.drop(["Year", "Source"], axis=1, inplace= True)
df.columns = [*df.columns[:-1], "Spending on education (% of GDP)"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


                    Country Spending on education (% of GDP)
0          Marshall Islands                             15.8
1                      Cuba                             12.9
2     Micronesia, Fed. Sts.                             12.5
3                  Kiribati                             12.0
4                   Somalia                              9.6
..                      ...                              ...
193                 Somalia                             n.a.
194  Bosnia and Herzegovina                             n.a.
195              Montenegro                             n.a.
196             North Korea                             n.a.
197        Papua New Guinea                             n.a.

[198 rows x 2 columns]


In [87]:
#35. List of sovereign states by homeless population

homeless_pop = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_sovereign_states_by_homeless_population", 1)

df = homeless_pop

df.drop(["Homeless (average day)", "Data year", "Unsheltered per 10k", "Main article, other notes", "Unnamed: 6"], axis=1, inplace=True)
df.columns = [*df.columns[:-1], "Homeless population per 10k"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


          Country  Homeless population per 10k
0     Afghanistan                       1180.0
1         Albania                        121.0
2       Australia                         48.0
3         Austria                         25.4
4      Azerbaijan                        725.0
..            ...                          ...
86  United States                         17.5
87        Vatican                          0.0
88        Vietnam                         16.6
89          Yemen                       1294.0
90       Zimbabwe                        848.0

[91 rows x 2 columns]


In [88]:
#36. List of countries by milk consumption per capita

milk_consumption = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_milk_consumption_per_capita", 1)

df = milk_consumption

df.drop(["Rank", "Change in rank 2013/2007", "Milk consumption 2007 (kg/capita/yr) [2]"], axis=1, inplace=True)
df.columns = [*df.columns[:-1], "Milk consumption per capita"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Netherlands Antilles not found in regex


         Country Milk consumption per capita
0        Finland                      430.76
1     Montenegro                      349.21
2    Netherlands                      341.47
3         Sweden                      341.23
4    Switzerland                      318.69
..           ...                         ...
178  North Korea                        3.79
179     Cambodia                        3.47
180         Laos                        2.92
181      Liberia                        3.04
182     DR Congo                            

[182 rows x 2 columns]


In [89]:
#37. List of countries by number of scientific and technical journal articles

sci_publi = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_number_of_scientific_and_technical_journal_articles", 1)

df = sci_publi

df.drop(["Rank", "Scientific publications per capita (per million)"], axis=1, inplace=True)
df.columns = [*df.columns[:-1], "Number of scientific publications"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
World not found in regex


                            Country  Number of scientific publications
0                             China                             744042
1                     United States                             624554
2                    United Kingdom                             198500
3                             India                             191590
4                           Germany                             174524
..                              ...                                ...
224  Svalbard and Jan Mayen Islands                                  3
225                          Tuvalu                                  2
226         St. Pierre and Miquelon                                  1
227                         Tokelau                                  1
228     French Southern Territories                                  1

[229 rows x 2 columns]


In [90]:
#38. Books published per country per year

books_publi = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Books_published_per_country_per_year", 1)

df = books_publi

df.drop(["Rank", "Year", "Notes", "References"], axis=1, inplace=True)
df.columns = [*df.columns[:-1], "Books published per year"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


            Country Books published per year
0     United States                   275232
1             China                   208418
2    United Kingdom                   186000
3             Japan                   139078
4         Indonesia                   135081
..              ...                      ...
128          Angola                       22
129          Gambia                       14
130            Mali                       14
131    Burkina Faso                       12
132            Oman                        7

[133 rows x 2 columns]


In [91]:
#39. List of countries by food energy intake

kilocal = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_food_energy_intake", 1)

df = kilocal

df.drop(["Rank", ('Average daily dietary energy consumption per capita[8]', 'Year')], axis=1, inplace=True)
df.columns = [*df.columns[:-1], "Average daily dietary energy consumption per capita kcal"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/3129925410.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(["Rank", ('Average daily dietary energy consumption per capita[8]', 'Year')], axis=1, inplace=True)
More than one regular expression match for New Caledonia (France)
More than one regular expression match for New Caledonia (France)
More than one regular expression match for New Caledonia (France)
More than one regular expression match for New Caledonia (France)
More than one regular expression match for New Caledonia (France)
More than one regular expression match for New Caledonia (France)


                      Country  \
0                     Ireland   
1               United States   
2                     Belgium   
3                     Türkiye   
4                     Austria   
..                        ...   
166                    Zambia   
167                    Uganda   
168                Madagascar   
169                  Zimbabwe   
170  Central African Republic   

     Average daily dietary energy consumption per capita kcal  
0                                                 3885         
1                                                 3782         
2                                                 3769         
3                                                 3711         
4                                                 3695         
..                                                 ...         
166                                               2002         
167                                               1981         
168                        

In [92]:
#40. List of countries by average yearly temperature

temp_celsius_avg = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature", 1)

df = temp_celsius_avg

df.columns = [*df.columns[:-1], "Average yearly temperature from 1961 to 1990 (°C)"]

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))


200


/var/folders/b9/hr3xjl8d091021r83fj044y40000gn/T/ipykernel_16493/2054447069.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


          Country Average yearly temperature from 1961 to 1990 (°C)
0    Burkina Faso                                             28.29
1            Mali                                             28.25
2        Kiribati                                             28.20
3        Djibouti                                             28.00
4        Maldives                                             28.00
..            ...                                               ...
187       Finland                                             01.55
188        Norway                                             01.50
189       Iceland                                             −0.70
190        Canada                                             −5.10
191        Russia                                             −5.35

[192 rows x 2 columns]


In [94]:
the_final_table = the_final_table.dropna(subset=['Country'])
the_final_table = the_final_table.drop_duplicates(subset='Country')
the_final_table = the_final_table.sort_values('Country')
the_final_table.to_csv('the_final_table.csv', index=False)

print(the_final_table.head(10000))


            Country World Bank[6]  Average download speed (Mbit/s)  \
285     Afghanistan           364                              NaN   
170         Albania          6803                            45.25   
202         Algeria          4274                              NaN   
144  American Samoa         15743                              NaN   
91          Andorra         41993                           191.23   
..              ...           ...                              ...   
308  Western Sahara           NaN                              NaN   
297     World total           NaN                              NaN   
278           Yemen           702                              NaN   
251          Zambia          1488                              NaN   
241        Zimbabwe          1267                              NaN   

    Alcohiol consumption per capita (litres) 2016  \
285                                           0.2   
170                                           7.5   
